<a href="https://colab.research.google.com/github/Elwing-Chou/tibame1224/blob/main/0114_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as req

url = "https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html"
response = req.urlopen(url)
response.read()

HTTPError: HTTP Error 403: Forbidden